In [1]:
# Imports
import duckdb
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

import sys
import os
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Agregate the dic to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
import utils

## Connection to formatted database
jdbc_url = 'jdbc:duckdb:./../data/explotation_zone/barcelona_processed.db'
driver = "org.duckdb.DuckDBDriver"

# SparkSession inicialitzation
spark = SparkSession.builder\
    .config("spark.jars", "./../lib/duckdb.jar") \
    .appName("DataExplotation") \
    .getOrCreate()

In [2]:
df_airbnb = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_airbnb_listings") \
  .load()

Py4JJavaError: An error occurred while calling o32.load.
: java.lang.ExceptionInInitializerError
	at org.duckdb.DuckDBConnection.newConnection(DuckDBConnection.java:48)
	at org.duckdb.DuckDBDriver.connect(DuckDBDriver.java:41)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:63)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:241)
	at

In [4]:
from pyspark.ml.feature import Bucketizer

In [5]:
bucketizer = Bucketizer(splits=[0, 50, 150, 250, float('Inf')],
                        inputCol='price',
                        outputCol='price_discretized')

In [6]:
df_airbnb = bucketizer.setHandleInvalid('keep').transform(df_airbnb)

NameError: name 'df_airbnb' is not defined

In [ ]:
df_airbnb = df_airbnb.drop(['price'])

In [ ]:
df_criminal = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_criminal_dataset") \
  .load()

In [ ]:
df_reviews = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_tripadvisor_reviews") \
  .load()

In [ ]:
df_locations = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_tripadvisor_locations") \
  .load()


In [ ]:
con = duckdb.connect(database='./../data/explotation_zone/barcelona_processed_emb.db', read_only=False)
con.close()

## Write on the tables 
print('Writting tables...')
jdbc_url = 'jdbc:duckdb:./../data/explotation_zone/barcelona_processed_emb.db'
driver = "org.duckdb.DuckDBDriver"

print('    - Criminal table')
df_criminal.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_criminal_dataset") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Aribnb table')
df_airbnb.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_airbnb_listings") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Tripadvisor Locations table')
df_locations.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_tripadvisor_locations") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

print('    - Tripadvisor Reviews table')
df_reviews.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "df_tripadvisor_reviews") \
    .option("driver", driver) \
    .mode("overwrite") \
    .save()

spark.stop()
con.close()